In [1]:
from tensorflow.python import keras
import tensorflow as tf
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

from tensorflow.python.keras import backend
backend.set_session(session)
print(backend._get_available_gpus())

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

['/job:localhost/replica:0/task:0/device:GPU:0']
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1014351117149352633
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14591099164818806848
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5399445504
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8841533958096816652
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13066002113158655287
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import Input, Model, load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.python.keras.layers import Dense, LSTM, Concatenate, Embedding, Bidirectional

from gensim.models import Word2Vec

from random import randint, choice

#import nltk
#nltk.download('averaged_perceptron_tagger')

import sys
sys.path.insert(1, '../utils')
from NLP_little_helpers import *
little_helpers()

Using TensorFlow backend.
[nltk_data] Downloading package brown to /home/felipe/nltk_data...
[nltk_data]   Package brown is already up-to-date!


potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, clean_entire, predict_tags, tagged_3D, tagged_n_grams, unknown_words_X, check_and_predict


In [3]:
from bs4 import BeautifulSoup

import requests
import pickle

import sys
sys.path.insert(1, '../utils')

from data_cleaning import clean_entire
from pos_tagging import predict_tags

In [ ]:
url = 'http://www.gutenberg.org/browse/scores/top'
response_url = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response_url.content, 'lxml')

book_id = [i.find('a').get('href').replace('/ebooks/', '') for i in soup.find_all('li') if i.find('a').get('href').startswith('/ebooks/')]
urls = ['http://www.gutenberg.org/cache/epub/'+i+'/pg'+i+'.txt' for i in book_id]

In [ ]:
books = list()

for url in urls:
    response_url = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response_url.content, 'lxml')
    books += [soup.find('p').get_text(' ', strip=True)]
    
print(len(books), 'books scraped.')

In [ ]:
#with open('books_and_datasets/100_books.txt', 'wb') as file:
#    pickle.dump(books, file)

In [4]:
with open('books_and_datasets/100_books.txt', 'rb') as file:
    books = pickle.load(file)[:100]

books[53] = ''
books[16] = ''
books[58] = ''
books[18] = ''
books[37] = ''
books[50] = ''
books[6] = ''
books[68] = ''
books[70] = ''
books[71] = ''
books[73] = ''
books[57] = ''
books[76] = ''
books[83] = ''
books[84] = ''
books[85] = ''
books[86] = ''
books[87] = ''
books[88] = ''
books[89] = ''
books[90] = ''
books[91] = ''
books[94] = ''
books[96] = ''
books[99] = ''

In [5]:
clean_text = list(set(clean_entire(' '.join(books))))

In [6]:
print(len(clean_text))
clean_text[:5]

495651


['',
 'but since hammerpurgstall and de sacy began to unwind the skein , many additional turns have been given',
 'thus , when herschel was prosecuting his beautiful observations on the other side of the channel , we had not even the means of verifying them',
 'as soon as the first brood had flown , and while they were yet under their parents care , they began to nest in one of the other boxes , the female as usual doing all the work and the male all the complimenting',
 'a young man , a farmlabourer , as come by us on his way to market with his masrs draysa journey of over five hundred mile , theer and backmade offers fur to take her fur his wife wives is very scarce theer , and then to set up fur their two selves in the bush']

In [7]:
with open('../Trained_Weights/hmm_model.pkl', 'rb') as file:
    hmm_model = pickle.load(file)
    
viterbis_dict = [predict_tags(text, hmm_model) for text in clean_text if len(text) > 20]

with open('../Data/100_books_tagged.json', 'w') as outfile:
    json.dump(viterbis_dict, outfile)